In [ ]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

import os
import time
import numpy as np
import pandas as pd
import neuropythy as ny
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.optimize import curve_fit

In [ ]:
roi_results = np.load('roi_results_area.npy')
roi_metrics = np.load('roi_metrics_area.npy')

In [ ]:
# Plotting CM vs ecc for all ROIs + Horton curve (ground truth)

# Single plot for all ROIs
fig, ax = plt.subplots(figsize=(14, 14))

# Generate rainbow colors
n_rois = len(roi_results)
colors = cm.rainbow(np.linspace(0, 1, n_rois))

for i, (roi, res) in enumerate(roi_results.items()):
    color = colors[i]

    # Make sure CI bounds are in correct order
    y_lower = np.minimum(res["y_lower"], res["y_upper"])
    y_upper = np.maximum(res["y_lower"], res["y_upper"])

    # Harvey fit
    ax.plot(res["x_common"], res["y_fixed"],
            lw=2, label=roi, color=color, zorder=3)

    # 95% prediction interval as translucent shadow
    ax.fill_between(
        res["x_common"], y_lower, y_upper,
        color=color, alpha=0.2, zorder=2, edgecolor=None
    )

# Plot Horton curve (once)
# Take the first ROI for reference
first_res = next(iter(roi_results.values()))
ax.plot(first_res["x_common"], first_res["y_horton"],
        'k--', lw=2, label="Horton", zorder=4)

# Axis labels and limits
ax.set_xlabel("Eccentricity (deg)", fontsize = 35)
ax.set_ylabel("Cortical magnification (mm²/deg²)", fontsize = 35)
ax.set_yscale("log")
ax.set_ylim(0, 1000)
ax.set_xticklabels(np.arange(0, 9, 1), fontsize = 25)  # enforce clean labels
ax.set_xlim(0, 8)
ax.tick_params(axis='y', labelsize=25)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Legend with Horton last
handles, labels = ax.get_legend_handles_labels()
if "Horton" in labels:
    idx = labels.index("Horton")
    handles.append(handles.pop(idx))
    labels.append(labels.pop(idx))
ax.legend(handles, labels, fontsize=18, ncol=2,)

plt.tight_layout()
plt.savefig("area_all_rois_curves_CI.png", dpi=300,
            bbox_inches='tight', facecolor='white')
plt.show()


In [ ]:
# RMSE of Harvey model to CM vs ecc data
rois = ["V1", "V2", "V3", "hV4", "VO1", "VO2",
        "PHC1", "PHC2", "TO2", "TO1", "LO2", "LO1", "V3B", "V3A",
        "IPS0", "IPS1", "IPS2", "IPS3", "IPS4", "IPS5", "SPL1", "FEF"]
n_rois = len(rois)

# Collect Harvey RMSE values
rmse_harvey = [roi_metrics[roi]['Harvey_RMSE'] for roi in rois]

x = np.arange(n_rois)
width = 0.6  # width of the bars

fig, ax = plt.subplots(figsize=(5, 4))

# Plot RMSE bars
ax.bar(x, rmse_harvey, width, color=colors, label='Harvey RMSE')

# Labels and title
ax.set_xticks(x)
ax.set_xticklabels(rois, rotation=90, fontsize=13)
ax.set_ylabel('RMSE', fontsize=20)
ax.set_xlabel('ROI', fontsize=20)
ax.tick_params(axis='y', labelsize=14)
# Remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig("area_roi_RMSE.png", dpi=300,
            bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
# hexbin V1
# plot raw values (hexbin V1 only)
hexbin_roi = grouptsv[grouptsv['roi'] == 'V1'].copy()
hexbin_roi = hexbin_roi.groupby('subject').filter(lambda g: len(g) > 2)
# Create figure and axis
fig, ax = plt.subplots(figsize=(5,4))

# Hexbin plot
hb = ax.hexbin(
    hexbin_roi['prf_ecc'], 
    np.maximum(hexbin_roi['pRF_CM'], 1e-6),
    gridsize=100, 
    cmap='Spectral', 
    mincnt=1, 
    alpha=0.8, 
    linewidths=0.2
)

ax.set_xlim(0, 8)
ax.set_xticklabels(np.arange(0, 9, 1), fontsize = 14)  # enforce clean labels
ax.tick_params(axis='y', labelsize=14)
ax.set_ylabel('CMF (mm²/deg²)', fontsize = 20)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.savefig("area_V1_CM_raw_plot.png", dpi=300, bbox_inches='tight')  # PNG
plt.show()

In [ ]:
# Median binned V1
import numpy as np
import matplotlib.pyplot as plt

# Subset V1 data
df_v1 = grouptsv[grouptsv['roi']=='V1']

# Eccentricity values
x = df_v1['prf_ecc'].to_numpy()
y = df_v1['pRF_CM'].to_numpy()

# Define non-uniform bins: denser near fovea, sparser in periphery
n_bins = 13
bin_edges = np.linspace(0.5, np.sqrt(x.max()), n_bins)**2
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # x-axis positions

# Assign each data point to a bin
bin_idx = np.digitize(x, bin_edges)

# Collect y values for each bin
binned_data = []
binned_x = []

for i in range(1, len(bin_edges)):
    mask = bin_idx == i
    if np.any(mask):
        binned_x.append(bin_centers[i-1])
        binned_data.append(y[mask])

# Plot
fig, ax = plt.subplots(figsize=(5,4))

ax.boxplot(
    binned_data,
    positions=binned_x,
    widths=0.2,   # <-- uniform width for all boxes
    patch_artist=True,
    boxprops=dict(facecolor="lightcoral", color="r"),
    medianprops=dict(color="black", linewidth=1.2),
    whiskerprops=dict(color="r"),
    capprops=dict(color="r"),
    showfliers=False   # no outlier dots
)

# Clean x-axis: show only 0–8
ax.set_xlim(0, 8)
ax.set_xticks(np.arange(0, 9, 1))   # tick locations
ax.set_xticklabels(np.arange(0, 9, 1), fontsize = 14)  # enforce clean labels

ax.set_xlabel('Eccentricity (deg)', fontsize = 20)
ax.set_ylabel('CMF (mm²/deg²)', fontsize = 20)
ax.tick_params(axis='y', labelsize=14)
#ax.set_title('V1: Cortical Magnification by Eccentricity Bins')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.savefig("area_V1_CM_bin_plot.png", dpi=300, bbox_inches='tight')  # PNG
plt.show()
